In [1]:
import os
import json
import time
import requests
import pandas as pd

from tqdm.notebook import tqdm

bearer_token = "AAAAAAAAAAAAAAAAAAAAADSXjAEAAAAAQ32sCpTWcilVX%2BQj0BjOArOSYCE%3DUax6ZCizLRwwQcWQirExGQEDEB903dxBZKM4LAuGTMTvMa4jVP"

## Parameters

In [2]:
SLEEP = 2
SAVE_ITER = 1000
FILE = "datasets/comms.csv"

In [3]:
twitter_comms = pd.read_csv("datasets/twitter_comms_dataset.csv")
waterloo = pd.read_csv("datasets/cards_waterloo.csv", low_memory=False)

In [4]:
twitter_comms = twitter_comms[~twitter_comms.tweet_id.isin(waterloo.tweetid)]

In [5]:
s = requests.Session()
def getTweetsByID(id_):
    # Twitter Endpoint
    url = f"https://api.twitter.com/2/tweets/{id_}"
    
    # Auth Bearer no research access or premium for the moment
    headers = {
        "Authorization": "Bearer {}".format(bearer_token)
    }
    
    # Query parameters
    params = {
        'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
        'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
        'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
        'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
        'next_token': {}
    }

    try:
        time.sleep(6)
        req = requests.Request(
            "GET",
            url,
            params=params,
            headers=headers
        ).prepare()

        response = s.send(req)
        results = json.loads(response.text)
        return results
    except:
        print(json.loads(response.text))

In [6]:
if os.path.isfile(FILE):
    tweets = pd.read_csv(FILE)
    tweets_to_process = list(set(twitter_comms.tweet_id.unique()) - set(tweets.id.unique()))
else:
    tweets = pd.DataFrame()
    tweets_to_process = twitter_comms.tweet_id.unique()

In [7]:
for i, id_ in tqdm(enumerate(tweets_to_process), total=len(tweets_to_process)):
    ## Get tweet from API
    response = getTweetsByID(id_)
    
    ## Preprocess response into the dataframe
    if "errors" not in response:
        sample = pd.DataFrame([response["data"]])
        sample["user"] = [response["includes"]["users"][0]]
    else:
        sample = pd.DataFrame([response["errors"]])
        sample["id"] = id_
    tweets = pd.concat([tweets, sample], axis=1)

    ## Save a copy every SAVE_ITER steps
    if i%SAVE_ITER==0:
        tweets.to_csv(FILE, index=False)

  0%|          | 0/884307 [00:00<?, ?it/s]

UnboundLocalError: local variable 'response' referenced before assignment

In [ ]:
response